In [1]:
Descricao = '''

Mistura de óleos:
Dois tipos de óleos vegetais: V1, V2,
Três tipos de óleos não vegetais: NV1, NV2, NV3.

Ao misturar os óleos é possível produzir um novo óleo. Tal óleo pode ser vendido por $150.

Restrições:
Máximo de 200 ton. de óleos vegetais / mês.
Máximo de 250 ton. óleos não vegetais / mês.
Acidez do óleo final deve estar entre 3 e 6.

Objetivo: maximizar o lucro.

Tabela:
columns = 'V1 V2 NV1 NV2 NV3'
index = 'Custo/ton Acidez'
r1 = '110 120 130 110 115'
r2 = '8.8 6.1 2.0 4.2 5.0'
'''

In [2]:
#Chamar livrarias necessárias para resolução
import numpy as np
import pandas as pd
import scipy.optimize
import pulp

In [3]:
#Criar tabela
columns = 'V1 V2 NV1 NV2 NV3'.split()
index = 'Custo/ton Acidez'.split()
custo = np.array([int(x) for x in '110 120 130 110 115'.split()])
acidez = np.array([float(x) for x in '8.8 6.1 2.0 4.2 5.0'.split()])

display(pd.DataFrame([custo, acidez], columns = columns, index = index))

,V1,V2,NV1,NV2,NV3
Custo/ton,110.0,120.0,130.0,110.0,115.0
Acidez,8.8,6.1,2.0,4.2,5.0


In [4]:
#Resolução por SciPy
z = 150 - custo
A = [-(acidez - 3), acidez - 6, np.array([1, 1, 0, 0, 0]), np.array([0, 0, 1, 1, 1])]
B = np.array([0, 0, 200, 250])
X = [(0, None)] * 5

print('z:', z, 'A:', A, 'B:', B, 'X:', X, sep = '\n'*2)

#Rodar otimizador:
scipy.optimize.linprog(-z, A_ub = A, b_ub = B, bounds = X, method = 'simplex', options={"disp": True})

z:

[40 30 20 40 35]

A:

[array([-5.8, -3.1,  1. , -1.2, -2. ]), array([ 2.8,  0.1, -4. , -1.8, -1. ]), array([1, 1, 0, 0, 0]), array([0, 0, 1, 1, 1])]

B:

[  0   0 200 250]

X:

[(0, None), (0, None), (0, None), (0, None), (0, None)]
Optimization terminated successfully.
         Current function value: -17592.592593
         Iterations: 4


     fun: -17592.592592592595
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([1350.,    0.,    0.,    0.])
  status: 0
 success: True
       x: array([159.25925926,  40.74074074,   0.        , 250.        ,
         0.        ])

In [5]:
#Solução: [159.25925926,  40.74074074,   0.        , 250.        ,0.        ]
#Máximo: 17592.592592592595

In [6]:
#Solução por PulP
lucro = pulp.LpProblem('Lucro', pulp.LpMaximize)

#Definir variáveis: quantidade de cada grão
x0 = pulp.LpVariable('x0', lowBound=0, cat='Continuous')
x1 = pulp.LpVariable('x1', lowBound=0, cat='Continuous')
x2 = pulp.LpVariable('x2', lowBound=0, cat='Continuous')
x3 = pulp.LpVariable('x3', lowBound=0, cat='Continuous')
x4 = pulp.LpVariable('x4', lowBound=0, cat='Continuous')

#Definir função objetivo
lucro += 40 * x0 + 30 * x1 + 20 * x2 + 40 * x3 + 35 * x4

#Definir restrições
lucro += 5.8 * x0 + 3.1 * x1 + -1 * x2 + 1.2 * x3 + 2 * x4 >= 0
lucro += 2.8 * x0 + 0.1 * x1 + 4 * x2 - 1.8 * x3 - 1 * x4 <= 0
lucro += 1 * x0 + 1 * x1 <= 200
lucro += 1 * x2 + 1 * x3 + 1 * x4 <= 250

display(lucro)

Lucro:
MAXIMIZE
40*x0 + 30*x1 + 20*x2 + 40*x3 + 35*x4 + 0
SUBJECT TO
_C1: 5.8 x0 + 3.1 x1 - x2 + 1.2 x3 + 2 x4 >= 0

_C2: 2.8 x0 + 0.1 x1 + 4 x2 - 1.8 x3 - x4 <= 0

_C3: x0 + x1 <= 200

_C4: x2 + x3 + x4 <= 250

VARIABLES
x0 Continuous
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous

In [7]:
#Resolução
lucro.solve()

pulp.LpStatus[lucro.status]

for variable in lucro.variables():
    print("{} = {}".format(variable.name, variable.varValue))

print(pulp.value(lucro.objective))

x0 = 159.25926
x1 = 40.740741
x2 = 0.0
x3 = 250.0
x4 = 0.0
17592.59263


In [8]:
#Mesma solução que SciPy